Importando bibliotecas necessárias para importar, tratar e salvar os dados do IPEA (via bibioteca do ipea em http://www.ipeadata.gov.br/Default.aspx) em csv

In [1]:
# iImportação das bibliotecas
import ipeadatapy as ip
import pandas as pd
from io import StringIO
from azure.storage.filedatalake import DataLakeServiceClient

In [2]:
# Importando dados do IPEA
codigo = 'EIA366_PBRENT366'

df = ip.timeseries(codigo)

In [3]:
# Transformando o dataframe importado em um dataframe do pandas
df = pd.DataFrame(df)

In [4]:
# Selecionando apenas a coluna de interesse e o intervalo de datas desejado
df = df.loc['2015-01-01':, 'VALUE (US$)']

In [5]:
# Trnsformando o panda series em um dataframe
df = pd.DataFrame(df)

In [6]:
# Renomeando a coluna para facilitar o uso
df = df.rename(columns={'VALUE (US$)': 'valor_us$'})

In [7]:
# Renomeando o índice para facilitar o uso
df.index.name = 'data'

In [8]:
# Transformando o indice em tipo data
df.index = pd.to_datetime(df.index)

In [9]:
# Formatando o índice para o formato desejado
df.index = df.index.strftime('%d/%m/%Y')

In [10]:
# Excluindo os valores nulos
df.dropna(inplace=True)

In [13]:
# Salvando o dataframe em um arquivo CSV no datalake do Azure

csv_buffer = StringIO()
df.to_csv(csv_buffer, index=True, header=True, sep=';', index_label='data', decimal=',', float_format='%.2f')

account_name = "datalaketech4"
filesystem_name = "dados-ipea"
path_in_datalake = "cotacao_petroleo_ipea.csv"

sas_token = "sp=rw&st=2025-04-26T20:14:36Z&se=2025-04-27T04:14:36Z&spr=https&sv=2024-11-04&sr=c&sig=23oj2eG5adqmroQgcMXFDPWXNMSfBpLl2LxBhmC344E%3D"

service_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net?{sas_token}"
)

file_system_client = service_client.get_file_system_client(file_system=filesystem_name)
file_client = file_system_client.get_file_client(path_in_datalake)

file_client.upload_data(csv_buffer.getvalue(), overwrite=True)


{'date': datetime.datetime(2025, 4, 26, 22, 31, 26, tzinfo=datetime.timezone.utc),
 'etag': '"0x8DD85121560AF94"',
 'last_modified': datetime.datetime(2025, 4, 26, 22, 31, 27, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': '30bc654e-22ee-11f0-80ad-74563cf125a2',
 'request_id': '0906bbf0-c01f-004b-3bfa-b66ee8000000',
 'version': '2025-05-05',
 'request_server_encrypted': False,
 'encryption_key_sha256': None,
 'lease_renewed': None}